In [ ]:
import os
import cv2
from cv2 import imread
from cv2 import imshow
from cv2 import CascadeClassifier
from cv2 import rectangle
import numpy as np
import pandas as pd
import imutils 
from imutils import paths
from skimage.feature import hog, greycomatrix, greycoprops
from skimage import data, segmentation, color, filters, exposure, restoration
from skimage.util import img_as_ubyte
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import black_tophat, skeletonize, convex_hull_image
from skimage.morphology import disk
import matplotlib.pyplot as plt

from scipy.signal import convolve2d as conv2
from scipy import ndimage as ndi

from sklearn import metrics
from skimage.data import gravel
from skimage.filters import difference_of_gaussians, window
from scipy.fftpack import fftn, fftshift
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score, f1_score, accuracy_score, precision_score
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split

from mtcnn.mtcnn import MTCNN
from matplotlib.patches import Rectangle
import tensorflow as tf
from zipfile import ZipFile
import pickle
from matplotlib import style
import seaborn as sns

# from skimage import data
# from skimage.color import rgb2gray


In [ ]:
with tf.device('/device:GPU:0'):
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    base_path = 'I:/Thesis/dataset/final/train_modi - copy'
    categories = ['angry' ,'happy', 'sad', 'surprised']


    def bgr2rgb(img):
        return cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)
    # resize image
    def image_to_feature_vector(images, size=(150, 150)):
        return cv2.resize(images, size).flatten()


    def make(rc_img):
        '''
        fig, axes = plt.subplots(ncols=3, figsize=(15, 3.5))
        ax = axes.ravel()
        ax[0] = plt.subplot(1, 3, 1)
        ax[1] = plt.subplot(1, 3, 2)
        ax[2] = plt.subplot(1, 3, 3)
        '''
        selem = disk(20)
        eroded = erosion(rc_img, selem)
        roi = rc_img - eroded


        pixels = image_to_feature_vector(roi)

        rawImages.append(pixels)                   # features
        labels.append(cat)
        
        '''
        ax[0].imshow(rc_img)
        ax[0].set_title('Original')
        ax[0].axis('off')
        
        ax[1].imshow(eroded, cmap=plt.cm.gray)
        ax[1].set_title('Segmented pic')
        ax[1].axis('off')

    #     ax[2].hist(out.ravel(),256,[0,256], color='r')
    #     ax[2].set_title('Histogram')

        ax[2].imshow(roi, cmap=plt.cm.gray)
        ax[2].axis('off')

        plt.show()
        '''

    labels = []
    rawImages = []
    
    c,b,f=0,0,0
#     c = total number of images
#     b = detected human face
#     f = human face not found

    for cat in categories:

        image_dir = os.path.join(base_path, cat)
        imagePaths = list(paths.list_images(image_dir))

        for (i, imagePath) in enumerate(imagePaths):
            c=c+1
            '''
            fig, axes = plt.subplots(ncols=3, figsize=(15, 3.5))
            ax = axes.ravel()
            ax[0] = plt.subplot(1, 3, 1)
            ax[1] = plt.subplot(1, 3, 2)
            ax[2] = plt.subplot(1, 3, 3)
            '''

            image = cv2.imread(imagePath)
#             img = image[:,:,::-1]
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


            # perform face detection
            bboxes = face_cascade.detectMultiScale(gray, 1.1, 5)
#             ax[2] = plt.gca()
            if list(bboxes):
                b=b+1

                # print bounding box for each detected face
                twidth, theight = 0, 0
                for box in bboxes:
                    x, y, width, height = box

                    if width > twidth and height > theight:
                        twidth, theight = width, height
                        x2, y2 = x + twidth, y + theight
                        # draw a rectangle over the pixels
                        roi_gray=gray[y:y2, x:x2]
#                         rectangle(gray, (x, y), (x2, y2), (0,255,0), 3)
#                         rect = Rectangle((x, y),twidth, theight, fill=False, color='yellow')
                    else:
                        continue
                  
                make(roi_gray)
                roi_flip = cv2.flip(roi_gray, 1)
                make(roi_flip)
#                 roi_rotate = cv2.rotate(roi_gray, cv2.cv2.ROTATE_90_CLOCKWISE)
#                 roi_rotate = imutils.rotate(roi_gray, angle=90)
#                 make(roi_rotate)

            else:
                f=f+1
                
        print(f"{cat}: c={c}, b={b}, f={f}")
    
    rawImages = np.array(rawImages)               # show some information on the memory consumed by the raw images matrix and features matrix
    labels = np.array(labels)

In [ ]:
rawImages.size

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(rawImages, labels, test_size=0.25, random_state=30)
#     (train_Feat, test_Feat, train_Labels, test_Labels) = train_test_split(features, labels, test_size=0.25, random_state=30)

# Classifier before tuning

In [ ]:
def train_model(model, name):
    model.fit(X_train,y_train)

    test_prediction = model.predict(X_test)
    train_prediction= model.predict(X_train)
    
    print("\n"+ name + "Classifier: \n")

    test_matrics = confusion_matrix(y_test, test_prediction)
    print("Confusion Matrix:\n", test_matrics)

    test_report = classification_report(y_test, test_prediction)
    print("Classification Report : \n", test_report)
    
    test_acc = accuracy_score(y_test, test_prediction)                     # test accuracy
    train_acc = accuracy_score(y_train, train_prediction)
    
    print("Accuracy of " + name + " : \n")
          
    print('Train Accuracy : {:.5f} %'.format(train_acc * 100))
    print('Test Accuracy : {:.5f} % \n'.format(test_acc * 100))
          
    print('Train Recall : ', recall_score(y_train, train_prediction, average='weighted'))
    print("Test Recall : ", recall_score(y_test, test_prediction, average='weighted'), '\n')
          
    print("Test F1 : ", f1_score(y_test, test_prediction, average='weighted'))
    print("Test precison : ", precision_score(y_test, test_prediction, average='weighted'))
    

    style.use('fivethirtyeight')
    sns.set(style='whitegrid', color_codes=True, font_scale=1.2)    # 'font_scale' = size of label
    
    plt.figure(figsize=(6,4))
    sns.heatmap(test_matrics, annot=True, annot_kws={"size": 17}, fmt = "d")
    plt.show()

    return test_acc

In [ ]:
# Decision Tree Classifier:............................................................................

name = "Decision Tree"
dt = DecisionTreeClassifier(criterion="entropy", max_depth=5)
dt_acc = train_model(dt, name)


# Logistic Regression Final:..........................................................................

name = "Logistic Regression"
lr = LogisticRegression()
lr_acc = train_model(lr, name)


# SVM Classifier:.....................................................................................

name = "SVM"
svm = SVC(kernel='linear')
svm_acc = train_model(svm, name)


# Random Forest Classifier:..................................................................................

name = "Random Forest"
rn = RandomForestClassifier(n_estimators=90)
rn_acc = train_model(rn, name)


# Naive Bayes Classifier:........................................................................

name = "Naive Bayes"
nb = GaussianNB()
nb_acc = train_model(nb, name)


# xgBoost classifier:.........................................................................

name = "xgBoots"
xgb = XGBClassifier(n_estimators=10,objective ='binary:logistic',colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10,)
xgb_acc = train_model(xgb, name)


# ADABoost Classifier:.......................................

name = " ADABoost"
adab = AdaBoostClassifier()
adab_acc = train_model(adab, name)


# GradientBoost Classifier:.......................................

name = "Gradient Boost"
gb = GradientBoostingClassifier()
gb_acc = train_model(gb, name)

# Knn Classifier:..................................................................................

name = "KNN"
neighbors = [5, 7, 9, 19,]
k_pred = []

for k in neighbors:
    print(f"\nKNN classifier ( k = {k})\n")
    knn = KNeighborsClassifier(n_neighbors= k)
    knn_acc = train_model(knn, name)
    k_pred.append(knn_acc)
    


In [ ]:
# Make a plot...........................................................................................

height = [dt_acc*100, lr_acc*100, rn_acc*100, nb_acc*100, svm_acc*100, xgb_acc*100, adab_acc*100,
          gb_acc*100]
bars = ('Decision tree', 'Logistic','Random', 'Naive Bayes','SVM', 'xGBoost', 'ADABoost',
        'GradientBoost')

y_pos = np.arange(len(bars))

plt.figure(figsize=(20, 12))
plt.bar(y_pos, height, color=['red', 'pink', 'palegreen', 'green', 'blue', 'yellow', 'brown','magenta'])
plt.xticks(y_pos, bars)
plt.title('Accuracy Comparision', size = 30)
plt.show()

# Classifier after tuning

In [ ]:
def test(model, name):
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    print("\n"+ name + " Classifier: \n")

    test_matrics = confusion_matrix(y_test, test_pred)
    print("Confusion Matrix:\n", test_matrics)

    test_report = classification_report(y_test, test_pred)
    print("Classification Report : \n", test_report)
    
    test_acc = accuracy_score(y_test, test_pred)                     # test accuracy
    train_acc = accuracy_score(y_train, train_pred)
    
    print("Accuracy of " + name + " : \n")
          
    print('Train Accuracy : {:.5f} %'.format(train_acc * 100))
    print('Test Accuracy : {:.5f} % \n'.format(test_acc * 100))

    print('Train recall: ', recall_score(y_train, train_pred, average='macro'))
    print("Test recall:", recall_score(y_test, test_pred, average='macro'), '\n')
    
    print("Test f1:", f1_score(y_test, test_pred, average='macro'))
    print("Test precison : ", precision_score(y_test, test_pred, average='weighted'))
    
    style.use('fivethirtyeight')
    sns.set(style='whitegrid', color_codes=True, font_scale=1.2)   # 'font_scale' = size of label
    
    plt.figure(figsize=(6,4))
    sns.heatmap(test_matrics, annot=True, annot_kws={"size": 17}, fmt = "d")
    plt.show()

    return test_acc

    
def save_model(model, name):
    with open(name, 'wb') as f:
        pickle.dump(model, f)
        print("model saved")

In [ ]:
# ******************** Decision TreeFinal ******************

name = "Decision Tree"
dt_estimator = DecisionTreeClassifier(criterion='entropy', max_depth=15, max_features=8,
                       random_state=1)
dt_model = dt_estimator.fit(X_train, y_train)

dt_acc = test(dt_model, name)
# save_model(dt_model, 'model_decision_tree.pickle')

In [ ]:
# ********************* Logistic Regression Final *****************************

name = "Logistic Regression"
lr_estimetor = LogisticRegression(C=0.6, solver='liblinear')

lr_model = lr_estimetor.fit(X_train, y_train)

lr_acc = test(lr_model, name)
# save_model(lr_model, 'model_logistic_regression.pickle')

In [ ]:
# ******************** SVM Final ******************

name = "SVM"
svm_estimetor = SVC(C=0.0001, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=7, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

svm_model = svm_estimetor.fit(X_train, y_train)

svm_acc = test(svm_model, name)
# save_model(svm_model, 'model_svm.pickle')

In [ ]:
# ******************** Random Forest Final ******************

name = "Random Forest"
rf_estimetor = RandomForestClassifier(max_depth=10, min_samples_leaf=4, n_estimators=94,
                       random_state=137)

rf_model = rf_estimetor.fit(X_train, y_train)

rf_acc = test(rf_model, name)
# save_model(rf_model, 'model_random_forest.pickle')

In [ ]:
# ******************** xgBoost Final ******************

name = "xgBoost"
xgb_estimetor = XGBClassifier(base_score=0.5, booster=None, class_weight='balanced',
        colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
        importance_type='gain', interaction_constraints=None, learning_rate=0.1, max_delta_step=0,
        max_depth=10, min_child_weight=1, monotone_constraints=None, n_estimators=50, n_jobs=1,
        nthread=1, num_parallel_tree=1, objective='multi:softprob', random_state=0, reg_alpha=0,
        reg_lambda=1, scale_pos_weight=None, silent=True, subsample=0.5, tree_method=None,
        validate_parameters=False, verbosity=None)

xgb_model = xgb_estimetor.fit(X_train, y_train)

xgb_acc = test(xgb_model, name)
# save_model(rf_model, 'model_xgBoost.pickle')

In [ ]:
# ******************** Gradient Boosting MachineFinal ******************

name = "Gradient Boosting Machine"
gbm_estimetor = GradientBoostingClassifier(learning_rate=0.2, max_depth=2, n_estimators=120)

gbm_model = gbm_estimetor.fit(X_train, y_train)

gbm_acc = test(gbm_model, name)
# save_model(gbm_model, 'model_gradient_boosting_machine.pickle')

In [ ]:
# ******************** ADABoost Final ******************

name = "ADABoost"
adab_estimetor = AdaBoostClassifier(n_estimators=120, random_state=10)

adab_model = adab_estimetor.fit(X_train, y_train)

adab_acc = test(adab_model, name)
# save_model(gbm_model, 'model_gradient_boosting_machine.pickle')

In [ ]:
# ******************** Naive Bayes Final ******************

name = "Naive Bayes"
nb_estimetor = GaussianNB(var_smoothing=0.01)

nb_model = nb_estimetor.fit(X_train, y_train)

nb_acc = test(nb_model, name)
# save_model(knn_model, 'model_knn.pickle')

In [ ]:
# ******************** KNN Final ******************

name = "KNN"
knn_estimetor = KNeighborsClassifier(metric='manhattan', n_neighbors=17)

knn_model = knn_estimetor.fit(X_train, y_train)

knn_acc = test(knn_model, name)
# save_model(knn_model, 'model_knn.pickle')

In [ ]:
# Make a plot...........................................................................................

height = [dt_acc*100, lr_acc*100, rn_acc*100, nb_acc*100, svm_acc*100, xgb_acc*100, adab_acc*100, gbm_acc*100]

bars = ('Decision tree', 'Logistic', 'Random', 'Naive Bayes','SVM', 'xGBoost', 'ADABoost', 'GradientBoost')

y_pos = np.arange(len(bars))

plt.figure(figsize=(20, 12))
plt.bar(y_pos, height, color=['red', 'pink', 'palegreen', 'green', 'blue', 'yellow', 'brown', 'magenta'])
plt.xticks(y_pos, bars)
plt.title('Accuracy Comparision', size = 30)
plt.show()

# Tuning process section

In [ ]:
def test_tuning(grid_tree.best_estimator_):
    train_pred = grid_tree.best_estimator_.predict(X_train)
    y_pred = grid_tree.best_estimator_.predict(X_test)

    print('Train Accuracy: ', accuracy_score(y_train, train_pred ))
    print("Test Accuracy:", accuracy_score(y_test, y_pred), '\n')
    print('Train recall: ', recall_score(y_train, train_pred, average='macro'))
    print("Test recall:", recall_score(y_test, y_pred, average='macro'), '\n')
    print("Test f1:", f1_score(y_test, y_pred, average='macro'))

# Decision Tree - OK

In [ ]:

decision_tree = DecisionTreeClassifier(random_state=1)

# creating our parameters to test
param_dict={'max_depth': [14,15,17], 'min_samples_leaf':range(1,30,2), 'criterion': ['gini','entropy'],
           'max_features': range(0,9)}

#create the instance of GridSearchCV using the recall metric for our scoring. 
grid_tree= GridSearchCV(decision_tree, param_dict, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

#fit the Gridsearch to our data
grid_tree.fit(X_train,y_train)

In [ ]:
print(grid_tree.best_score_)
print(grid_tree.best_params_)
print(grid_tree.best_estimator_)
print("n")

In [ ]:
test_tuning(grid_tree.best_estimator_)

model_dt = grid_tree.best_estimator_

# Logistic Regression - OK

In [ ]:
# Logistic Regression*****************************

with tf.device('/device:GPU:0'):

    lr = LogisticRegression()  # instantiate the model object
    #'newton-cg', 'lbfgs', 'liblinear'', 'sag', 'saga'
    param_dict={'solver':[ 'lbfgs', 'liblinear', 'sag', 'saga'], 'C': [0.5, 0.6, 0.7], 
               'penalty': ['l1', 'l2', 'elasticnet']}  # set parameters to be searched through GridSearch

    #create the instance of GridSearchCV using the recall metric for our scoring. 
    grid_tree= GridSearchCV(lr, param_dict, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
    #fit the Gridsearch to our data
    grid_tree.fit(X_train,y_train)
  

In [ ]:
print(grid_tree.best_score_)  # outputs the best metric score
print(grid_tree.best_params_)  # outputs the parameters with best result
print(grid_tree.best_estimator_)  # outputs the model with the best parameters

In [ ]:
test_tuning(grid_tree.best_estimator_)

# SVM - OK

In [ ]:
# SVM ******************************** 'sigmoid', 'precomputed'

svm = SVC()
# defining parameter range 
param_grid = {'C': [0.0001, 0.001, 0.01], 'gamma': [8,7,6], 'kernel': ['rbf','linear','poly']}

grid_tree=GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
svm_model = grid_tree.fit(X_train, y_train)

In [ ]:
print(grid_tree.best_score_)
print(grid_tree.best_params_)
print(grid_tree.best_estimator_)

In [ ]:
test_tuning(grid_tree.best_estimator_)

# Random Forest - OK

In [ ]:
rfcc = RandomForestClassifier(random_state=137)
param_grid = {'max_depth':[8,9,10], 'n_estimators': [90,94,97 ], 
              'max_features': ['auto', 'log2', 'sqrt'], 'criterion': [ 'gini', 'entropy'],
              'min_samples_split': [ 2,3,4,5],'min_samples_leaf': [4,5,8]}

grid_tree=GridSearchCV(rfcc, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
model = grid_tree.fit(X_train, y_train)

In [ ]:
print(grid_tree.best_score_)
print(grid_tree.best_params_)
print(grid_tree.best_estimator_)

In [ ]:
test_tuning(grid_tree.best_estimator_)

In [ ]:
print("Classification Report: \n\n", classification_report(y_test, y_pred))
print("Confusion Matrix: \n", metrics.confusion_matrix(y_test, y_pred))

# xgBoost - OK

In [ ]:
xgb = XGBClassifier(class_weight='balanced', objective='binary:logistic',silent=True, nthread=1)
# A parameter grid for XGBoost
param_grid = {
        'min_child_weight': [1], 'subsample': [0.5],'colsample_bytree': [0.8],
        'max_depth': [10], 'learning_rate': [0.1],'n_estimators':[50],
        # n_estimators=10, learning_rate= 0.02, 'gamma': [ 1],
        }
grid_tree=GridSearchCV(xgb, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_tree.fit(X_train, y_train)

In [ ]:
print(grid_tree.best_score_)
print(grid_tree.best_params_)
print(grid_tree.best_estimator_)

In [ ]:
test_tuning(grid_tree.best_estimator_)

# Gradient Boosting Machine - OK

In [ ]:
gb = GradientBoostingClassifier()
param_grid = [{'n_estimators': [120], 'learning_rate':[0.2],'max_depth': [2],
                   'min_samples_split': [2]}]

grid_tree=GridSearchCV(gb, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_tree.fit(X_train, y_train.ravel())

In [ ]:
print(grid_tree.best_score_)
print(grid_tree.best_params_)
print(grid_tree.best_estimator_)

In [ ]:
test_tuning(grid_tree.best_estimator_)
gb = grid_tree.best_estimator_

# ADABoost - OK

In [ ]:

ada = AdaBoostClassifier()
param_grid = {'n_estimators': [100,120], 'algorithm': ['SAMME','SAMME.R'],
              'learning_rate': [1.0], 'random_state':[10,20]}
grid_tree=GridSearchCV(ada, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_tree.fit(X_train, y_train)

In [ ]:
print(grid_tree.best_score_)
print(grid_tree.best_params_)
print(grid_tree.best_estimator_)

In [ ]:
test_tuning(grid_tree.best_estimator_)

# Naive Bayes - OK

In [ ]:
with tf.device('/device:GPU:0'):
    gnb = GaussianNB()
    # defining parameter range 
    param_grid = {'var_smoothing':[0.01,0.1,0.2]}
    grid_tree=GridSearchCV(gnb, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
    grid_tree.fit(X_train, y_train)

In [ ]:
print(grid_tree.best_score_)
print(grid_tree.best_params_)
print(grid_tree.best_estimator_)

test_tuning(grid_tree.best_estimator_)
nb= grid_tree.best_estimator_

# KNN - OK

In [ ]:
knn = KNeighborsClassifier()
param_dict = {'n_neighbors': range(3,30, 2), 'metric': ['manhattan','euclidean','minkowski'],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}
grid_tree= GridSearchCV(knn, param_dict, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_tree.fit(X_train, y_train)

In [ ]:
print(grid_tree.best_score_)
print(grid_tree.best_params_)
print(grid_tree.best_estimator_)

In [ ]:
test_tuning(grid_tree.best_estimator_)
kn= grid_tree.best_estimator_